# EventinatR usage

The following shows a simplified usage of the EventinatR objects.

In [ ]:
#!pwsh
Remove-Item -Path ~\.nuget\packages\eventinatr\ -Recurse
dotnet pack ..\..\EventinatR.sln --output C:\Windows\Temp\EventinatR

In [ ]:
#i "nuget:C:\Windows\Temp\EventinatR"
#r "nuget:EventinatR"
#r "nuget:System.Memory.Data"
#r "nuget:System.Linq.Async"

using System;
using System.Collections.Generic;
using System.Linq;
using EventinatR;
using EventinatR.InMemory;

# Domain model

We are modeling a simple group with members, with the ability to add and remove members and the events that change the state of the group.

In [ ]:
public record GroupMember(string Name);
public record GroupId(string Name);

public abstract record Event;

public abstract record GroupEvent : Event
{
    public record Created(GroupId Id) : GroupEvent;
    public record AddedMember(GroupId Id, GroupMember Member) : GroupEvent;
    public record RemovedMember(GroupId Id, GroupMember Member) : GroupEvent;
}

public class Group
{
    private record GroupState(GroupId Id, IEnumerable<GroupMember> Members);

    public GroupId Id { get; private set; }
    public IEnumerable<GroupMember> Members => _members.AsEnumerable();

    private readonly List<GroupMember> _members = new();
    private readonly List<GroupEvent> _uncommittedEvents = new();

    private Group()
    {
    }

    private void AddEvent(GroupEvent e)
    {
        ApplyEvent(e);
        _uncommittedEvents.Add(e);
    }

    private void ApplyEvent(GroupEvent e)
    {
        switch (e)
        {
            case GroupEvent.Created created:
                Apply(created);
                break;
            case GroupEvent.AddedMember addedMember:
                Apply(addedMember);
                break;
            case GroupEvent.RemovedMember removedMember:
                Apply(removedMember);
                break;
            default:
                throw new InvalidOperationException($"Unsupported event: {e.GetType().FullName}");
        }
    }

    public static Group Create(string name)
    {
        var group = new Group();
        group.AddEvent(new GroupEvent.Created(new GroupId(name)));
        return group;
    }

    private void Apply(GroupEvent.Created e)
        => Id = e.Id;

    public void AddMember(string name)
    {
        if (!_members.Any(x => x.Name == name))
        {
            var member = new GroupMember(name);
            var e = new GroupEvent.AddedMember(Id, member);
            AddEvent(e);
        }
    }

    private void Apply(GroupEvent.AddedMember e)
        => _members.Add(e.Member);

    public void RemoveMember(string name)
    {
        var member = _members.FirstOrDefault(x => x.Name == name);

        if (member is not null)
        {
            var e = new GroupEvent.RemovedMember(Id, member);
            AddEvent(e);
        }
    }

    private void Apply(GroupEvent.RemovedMember e)
        => _members.Remove(e.Member);

    public static async Task<Group?> ReadAsync(EventStream stream)
    {
        var group = new Group();
        var snapshot = await stream.ReadSnapshotAsync<GroupState>();
        var state = snapshot.State;

        if (state is not null)
        {
            group.Id = state.Id;
            group._members.AddRange(state.Members);
        }

        var events = await snapshot.ReadAsync().ToListAsync();

        if (!events.Any() && state is null)
        {
            return null;
        }

        foreach (var e in events)
        {
            if (!e.TryConvert<GroupEvent>(out var groupEvent))
            {
                throw new InvalidOperationException($"The event {e.Data} is not supported.");
            }

            group.ApplyEvent(groupEvent);
        }

        return group;
    }

    public async Task SaveAsync(EventStream stream)
    {
        if (_uncommittedEvents.Any())
        {
            var state = new GroupState(Id, _members);
            _ = await stream.AppendAsync(_uncommittedEvents, state);
            _uncommittedEvents.Clear();
        }
    }
}


# Saving events to the stream

1. Initialize an in-memory event store
2. Create the aggregate root and perform some commands to generate events
3. Get a stream for the group from the event store
4. Save the events to the stream

In [ ]:
EventStore store = new InMemoryEventStore();

var group = Group.Create("Final Fantasty Games");

group.AddMember("Final Fantasy I");
group.AddMember("Final Fantasy II");
group.AddMember("Final Fantasy III");
group.AddMember("Final Fantasy IV");
group.AddMember("Final Fantasy V");
group.AddMember("Final Fantasy VI");
group.AddMember("Final Fantasy VII");
group.AddMember("Final Fantasy VIII");
group.AddMember("Final Fantasy IX");
group.AddMember("Final Fantasy X");
group.AddMember("Final Fantasy XI");
group.AddMember("Final Fantasy XII");
group.AddMember("Final Fantasy XIII");
group.AddMember("Final Fantasy XIV");
group.AddMember("Final Fantasy XV");

var stream = await store.GetStreamAsync(group.Id.Name);

await group.SaveAsync(stream);

display(group);

# Reading events from a stream

The aggregate root was designed to be able to read from a stream and get back to current state, the output should be exactly what was rendered in the previous step.

In [ ]:
group = await Group.ReadAsync(stream);
display(group);

# Generate some more events

Final Fantasy XIV was never really a game until A Realm Reborn was released.

In [ ]:
group.RemoveMember("Final Fantasy XIV");
group.AddMember("Final Fantasty XIV - A Realm Reborn");
await group.SaveAsync(stream);

display(group);

# Show the events from a stream

You can see each version, the type of event, and the event data.

In [ ]:
var events = await stream.ReadAsync().ToListAsync();
display(events);